# Create Heatmaps for significant IL8- and CXCR2-mediated signaling events

This notebook looks at the significant genes in at least one cancer. Pancancer heatmaps are created with circle size showing significance and color showing differences in median.

In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

import plot_utils as p

# Step 1: Run GSEA for significant genes in at least 1 cancer

First read in sig_pval_all_proteins.csv into a df. This csv file contains only genes with a significant p-value in at least one cancer. Then run GSEA using a list of genes from the df.

In [9]:
#root = R'~\Github\WhenMutationsDontMatter\PTEN\Step_3_trans_effect\csv'
#sig_df = pd.read_csv(root+R'/sig_pval_heatmap.csv')
sig_df = pd.read_csv("../../csv/sig_pval_heatmap.csv")
prot_list = list(sig_df.Proteomics) # list of genes with a sig pval in >= 1 cancer
prot_enr = gp.enrichr(gene_list = prot_list, description='Tumor_partition', gene_sets='NCI-Nature_2016', 
                       outdir='Enrichr')

In [3]:
prot_enr.res2d.head()

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,Regulation of RAC1 activity Homo sapiens 351aa...,20/38,1.112561e-08,0.000002,0,0,3.895750,71.346824,DOCK6;ARHGAP9;VAV3;RASGRF2;RAP1GDS1;VAV1;DEF6;...,NCI-Nature_2016
1,Class I PI3K signaling events Homo sapiens 12b...,19/48,6.480058e-06,0.000677,0,0,2.929928,35.003214,HSP90AA1;SYK;PLEKHA1;PDPK1;PTEN;PIK3CD;PIK3R1;...,NCI-Nature_2016
2,ATR signaling pathway Homo sapiens 8991cbac-61...,15/39,8.941964e-05,0.006230,0,0,2.846894,26.539231,RFC5;RFC3;TIPIN;RFC4;MCM7;RFC2;PLK1;RPA1;FANCD...,NCI-Nature_2016
3,E2F transcription factor network Homo sapiens ...,22/72,1.368548e-04,0.007151,0,0,2.261699,20.121410,PRMT5;RANBP1;RRM1;CDKN1B;CBX5;RRM2;APAF1;HDAC1...,NCI-Nature_2016
4,BCR signaling pathway Homo sapiens acbf44e2-61...,20/64,1.935528e-04,0.008091,0,0,2.313101,19.776925,MAP4K1;MAP2K1;SYK;CSNK2A1;CHUK;PDPK1;PTEN;PIK3...,NCI-Nature_2016


# Step 2: Get the list of significant genes 

In [10]:
index = 8
trans = prot_enr.res2d.Genes[index]
genes = trans.split(';')
print(prot_enr.res2d.Term[index])
print('total genes:',len(genes))

IL8- and CXCR2-mediated signaling events Homo sapiens fe78e284-6193-11e5-8ac5-06603eb7f303
total genes: 13


# Step 3: Create HeatMap

Slice out genes from the DNA Replication pathway from the df with genes sig in > 1 cancer.

In [11]:
# sig > 1 cancer
bool_df = sig_df.Proteomics.isin(genes)
plot_df = sig_df[bool_df]
len(plot_df.Proteomics.unique())

13

In [12]:
# Only include p-values < 0.01
plot_df = plot_df.loc[plot_df['P_Value'] <= 0.01]

In [13]:
p.plotCircleHeatMap(plot_df, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width= 800)

Loading BokehJS ...

# Step 4: Create a HeatMap with both +/- differences in median

 Read in the pos_neg_df.csv to create a df with only genes that have a pos and neg difference in median in different cancers. Slice out the genes that have a pos and neg difference in median in the pathway using the list of genes with a significant p-value in the pathway. 

In [8]:
pos_neg_df = pd.read_csv(root+R'\pos_neg_df.csv')

In [9]:
get = pos_neg_df.Proteomics.isin(genes) # bool df where True has both pos and neg
genes_pn = pos_neg_df[get] # Keep only genes with pos and neg
genes_pn.Proteomics.unique()

array(['ARRB1'], dtype=object)

In [10]:
# Only include p-values < 0.01
genes_pn = genes_pn.loc[genes_pn['P_Value'] <= 0.01]

In [11]:
p.plotCircleHeatMap(genes_pn, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width=600)

Loading BokehJS ...

# Sig in multiple cancers

In [12]:
mult_sig_df = pd.read_csv(root+R'\mult_sig_pval_heatmap.csv')

mult_sig_list = list(mult_sig_df.Proteomics.unique()) # list of genes with a sig pval in > 1 cancer
enr2 = gp.enrichr(gene_list = mult_sig_list, description='Tumor_partition', gene_sets='NCI-Nature_2016', 
                       outdir='/Enrichr')

In [13]:
enr2.res2d.head()

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,Class I PI3K signaling events Homo sapiens 12b...,10/48,3.923280e-08,0.000008,0,0,10.137875,172.888814,ZAP70;RAP1A;SYK;PIK3CA;PTEN;BLNK;PLCG2;BTK;PIK...,NCI-Nature_2016
1,BCR signaling pathway Homo sapiens acbf44e2-61...,10/64,6.779369e-07,0.000071,0,0,7.603406,108.000392,MAP4K1;PPP3CB;PTPRC;SYK;CHUK;PIK3CA;PTEN;BLNK;...,NCI-Nature_2016
2,ATR signaling pathway Homo sapiens 8991cbac-61...,8/39,1.046196e-06,0.000073,0,0,9.981908,137.454359,RFC5;RFC3;RFC4;MCM7;RFC2;FANCD2;TOPBP1;MCM2,NCI-Nature_2016
3,Aurora B signaling Homo sapiens 304a75af-618c-...,8/39,1.046196e-06,0.000055,0,0,9.981908,137.454359,NSUN2;NCL;NCAPG;NCAPD2;SMC4;NCAPH;NDC80;SMC2,NCI-Nature_2016
4,IL8- and CXCR2-mediated signaling events Homo ...,7/34,4.898355e-06,0.000205,0,0,10.018606,122.493600,GNG2;PRKCB;RAC2;ELMO1;DOCK2;PLCB2;GNAI2,NCI-Nature_2016


In [14]:
index2 = 4
trans2 = enr2.res2d.Genes[index2]
genes_mult = trans2.split(';')
print(enr2.res2d.Term[index2])
print('total genes:',len(genes_mult))

IL8- and CXCR2-mediated signaling events Homo sapiens fe78e284-6193-11e5-8ac5-06603eb7f303
total genes: 7


In [15]:
bool_df = mult_sig_df.Proteomics.isin(genes_mult)
plot_df2 = mult_sig_df[bool_df]
plot_df2.Proteomics.unique()

array(['ELMO1', 'DOCK2', 'PRKCB', 'PLCB2', 'GNAI2', 'GNG2', 'RAC2'],
      dtype=object)

In [16]:
# Only include p-values < 0.01
plot_df2 = plot_df2.loc[plot_df2['P_Value'] <= 0.01]

In [17]:
p.plotCircleHeatMap(plot_df2, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width=500)

Loading BokehJS ...

In [18]:
get = pos_neg_df.Proteomics.isin(genes_mult) # bool df where True has both pos and neg
genes_pn_mult = pos_neg_df[get] # Keep only genes with pos and neg
genes_pn_mult.Proteomics.unique()

array([], dtype=object)

In [19]:
#p.plotCircleHeatMap(genes_pn_mult, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
#                    plot_height=400, plot_width=600)